In [28]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('../data/checking-logs.sqlite')


In [29]:
sql = """
DROP TABLE IF EXISTS datamart;
CREATE TABLE datamart AS
WITH first_commits AS (
    SELECT uid, labname, MIN(timestamp) AS first_commit_ts
    FROM checker
    WHERE status='ready'
      AND numTrials=1
      AND labname IN ('laba04','laba04s','laba05','laba06','laba06s','project1')
      AND uid LIKE 'user_%'
    GROUP BY uid, labname
),
first_views AS (
    SELECT uid, MIN(datetime) AS first_view_ts
    FROM pageviews
    WHERE uid LIKE 'user_%'
    GROUP BY uid
)
SELECT fc.uid,
       fc.labname,
       fc.first_commit_ts,
       fv.first_view_ts
FROM first_commits fc
LEFT JOIN first_views fv ON fc.uid = fv.uid;
"""
conn.executescript(sql)


In [30]:
dm = pd.read_sql("SELECT * FROM datamart;", conn,
                 parse_dates=['first_commit_ts','first_view_ts'])

test = dm[dm.first_view_ts.notna()].copy()
control = dm[dm.first_view_ts.isna()].copy()

avg_view = test.first_view_ts.mean()
control['first_view_ts'] = avg_view

test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

test.head(), control.head()


(      uid  labname            first_commit_ts              first_view_ts
 0  user_1   laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
 1  user_1  laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
 2  user_1   laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
 3  user_1   laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
 4  user_1  laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136,
         uid   labname            first_commit_ts                 first_view_ts
 12  user_11    laba05 2020-05-03 21:06:55.970293 2020-04-27 00:40:05.761783552
 13  user_11  project1 2020-05-03 23:45:33.673409 2020-04-27 00:40:05.761783552
 14  user_12    laba04 2020-04-18 17:07:51.767358 2020-04-27 00:40:05.761783552
 15  user_12   laba04s 2020-04-26 15:42:38.070593 2020-04-27 00:40:05.761783552
 16  user_12    laba05 2020-05-03 08:39:25.174316 2020-04-27 00:40:05.761783552)

In [ ]:
datamart = pd.read_sql("SELECT * FROM datamart;", conn,
                       parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.info() # в чеклисте опечатки вроде как

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [32]:
control = pd.io.sql.read_sql('SELECT * FROM control', conn)

In [33]:
test = pd.io.sql.read_sql('SELECT * FROM test', conn)

In [34]:
conn.close()


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              59 non-null     object
 1   labname          59 non-null     object
 2   first_commit_ts  59 non-null     object
 3   first_view_ts    59 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB


In [36]:
control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              81 non-null     object
 1   labname          81 non-null     object
 2   first_commit_ts  81 non-null     object
 3   first_view_ts    81 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB
